In [1]:
interp.resolvers() = interp.resolvers() :+ ammonite.runtime.tools.Resolver.File(
  "m2",
  "/.m2/repository",
  "/[organisation]/[module]/[revision]/[artifact]-[revision].[ext]",
  m2 = true
)

In [2]:
import ammonite.ops._
import java.nio.file.Paths

// required spark dependencies
import $ivy.`org.apache.spark::spark-sql:2.1.0`
import $ivy.`org.apache.spark::spark-mllib:2.1.0`
import $ivy.`edu.berkeley.cs.amplab::keystoneml:0.4.0`
// required emma dependencies
import $ivy.`org.emmalanguage:emma-language:0.2-SNAPSHOT`
import $ivy.`org.emmalanguage:emma-spark:0.2-SNAPSHOT`

// project classpath
interp.load.cp(pwd / up / "emma-lib" / "target" / "classes")

import ammonite.ops._

import java.nio.file.Paths

// required spark dependencies

import $ivy.$                                  

import $ivy.$                                    

import $ivy.$                                         
// required emma dependencies

import $ivy.$                                            

import $ivy.$                                         

// project classpath


In [3]:
// spark-sql
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.Encoder
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.explode
// spark-mllib
import org.apache.spark.ml.feature.RegexTokenizer
import org.apache.spark.ml.feature.Tokenizer
import org.apache.spark.ml.feature.NGram
// emma-lib-base
import org.emmalanguage.data.reddit.Comment
import org.emmalanguage.api._
import org.emmalanguage.api.Meta.Projections._
import org.emmalanguage.api.spark._
import org.emmalanguage.lib.ml._
import org.emmalanguage.lib.ml.feature._

import org.apache.spark.sql.Dataset

import org.apache.spark.sql.Encoder

import org.apache.spark.sql.SparkSession

import org.apache.spark.sql.functions.explode
// spark-mllib

import org.apache.spark.ml.feature.RegexTokenizer

import org.apache.spark.ml.feature.Tokenizer

import org.apache.spark.ml.feature.NGram
// emma-lib-base

import org.emmalanguage.data.reddit.Comment

import org.emmalanguage.api._

import org.emmalanguage.api.Meta.Projections._

import org.emmalanguage.api.spark._

import org.emmalanguage.lib.ml._

import org.emmalanguage.lib.ml.feature._

In [4]:
// implicit Spark environment
implicit val spark = SparkSession.builder()
    .appName("N-Gram transformation")
    .master("local[*]")
    .config("spark.sql.warehouse.dir", Paths.get(sys.props("java.io.tmpdir"), "spark-warehouse").toUri.toString)
    .getOrCreate()

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/alexander/.coursier/cache/v1/https/repo1.maven.org/maven2/org/slf4j/slf4j-log4j12/1.7.16/slf4j-log4j12-1.7.16.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/alexander/.coursier/cache/v1/https/repo1.maven.org/maven2/org/slf4j/slf4j-log4j12/1.7.25/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


spark: SparkSession = org.apache.spark.sql.SparkSession@4753b042

In [5]:
import spark.implicits._

import spark.implicits._

In [6]:
val reddit = spark.read
    .schema(implicitly[Encoder[Comment]].schema)
    .json("file:///data/alexander/reddit/bz2/RC_2006-01.bz2")
    .as[Comment]

reddit: Dataset[Comment] = [author: string, author_flair_css_class: string ... 16 more fields]

In [7]:
val tokenizer = new RegexTokenizer()
  .setInputCol("body")
  .setOutputCol("tokens")
val tokenized = tokenizer.transform(reddit.filter($"id" === "c2722"))
tokenized.select("body", "tokens").as[(String, Array[String])].collect()

tokenizer: RegexTokenizer = regexTok_1c02b008d3f8
tokenized: org.apache.spark.sql.package.DataFrame = [author: string, author_flair_css_class: string ... 17 more fields]
res6_2: Array[(String, Array[String])] = Array(
  (
    "I have nothing but good things to say about Dell Tech Support. Many a time I've called in a faulty part and had the replacement at the front door in two days with a box to ship the old one back. First class service if you ask me.",
    Array(
      "i",
      "have",
      "nothing",
      "but",
      "good",
      "things",
...

In [11]:
val xs = DataBag.from(reddit)
val rs = emma.onSpark {
    split.gaps[Comment, String]()(_.id, _.body)(xs.withFilter(_.id == "c2722"))
}

xs: DataBag[Comment] = org.emmalanguage.api.SparkDataset@9c8c8073
rs: DataBag[FPoint[String, Array[String]]] = org.emmalanguage.api.SparkDataset@945cabed

In [12]:
rs.collect()

res11: Seq[FPoint[String, Array[String]]] = Array(
  FPoint(
    "c2722",
    Array(
      "I",
      "have",
      "nothing",
      "but",
      "good",
      "things",
      "to",
      "say",
...

In [ ]:
val ngramXfrm = new NGram().setN(3).setInputCol("body").setOutputCol("ngrams")
val ngrams = ngramXfrm.transform(reddit)